In [16]:
import sys 
import joblib
from tqdm import tqdm

BASEDIR = '..'
sys.path.insert(0, BASEDIR)

from src.db_drivers.vector_driver import VectorDriverConfig, VectorDBConnectionConfig, EmbedderModelConfig
from src.kg_model import EmbeddingsModelConfig, EmbeddingsModel, KnowledgeGraphModel

In [2]:
# Vector model configuration
NODES_DB_PATH = f'embeddings_part'
TRIPLETS_DB_PATH = f'embeddings_part'
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(
    path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR, db_info={'db': 'personalaidb', 'table': "vectorized_nodes"}))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(
    path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR, db_info={'db': 'personalaidb', 'table': "vectorized_triplets"}))

In [3]:
DEVICE = 'cuda'
EMBEDDER_MODEL_PATH = f'{BASEDIR}/models/intfloat/multilingual-e5-small'
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

In [4]:
VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

In [5]:
embd_model = EmbeddingsModel(VECTOR_MODEL_CONFIG)

No sentence-transformers model found with name ../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [6]:
triplets = joblib.load("../data/knowledge_graphs/diaasqa/gigachat_full/extracted_triplets")

In [11]:
embd_model.count_items()

{'nodes': 0, 'triplets': 0}

In [12]:
for triplet_pack in tqdm(triplets):
    embd_model.create_triplets(triplet_pack, status_bar=False)

100%|██████████| 3483/3483 [41:22<00:00,  1.40it/s] 


In [13]:
embd_model.count_items()

{'nodes': 49597, 'triplets': 44328}

In [19]:
base_graph_config = joblib.load("/home/dzigen/Desktop/PersonalAI/Personal-AI/data/knowledge_graphs/diaasqa/gigachat_full/graph_config")
base_embed_config = joblib.load("/home/dzigen/Desktop/PersonalAI/Personal-AI/data/knowledge_graphs/diaasqa/gigachat_full/embeddings_config")

# !!! IMPORTANT !!!
base_graph_config.driver_config.db_config.need_to_clear = False
base_embed_config.nodesdb_driver_config.db_config.need_to_clear = False
base_embed_config.tripletsdb_driver_config.db_config.need_to_clear = False
# !!! IMPORTANT !!!

# fixing paths to vector dbs
base_embed_config.embedder_config.model_name_or_path = '/'.join(base_embed_config.embedder_config.model_name_or_path.split("/")[2:])
base_embed_config.nodesdb_driver_config.db_config.path = '/'.join(base_embed_config.nodesdb_driver_config.db_config.path.split("/")[2:])
base_embed_config.tripletsdb_driver_config.db_config.path = '/'.join(base_embed_config.tripletsdb_driver_config.db_config.path.split("/")[2:])

In [21]:
base_kg_model = KnowledgeGraphModel(
    graph_config=base_graph_config,
    embeddings_config=base_embed_config)

No sentence-transformers model found with name ../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [22]:
base_kg_model.count_items()

{'graph_info': {'triplets': 182838, 'nodes': 49597},
 'embeddings_info': {'nodes': 49597, 'triplets': 44328}}

In [23]:
tmp_node_ids = base_kg_model.graph_struct.db_conn.execute_query("MATCH (a) RETURN a.str_id as str_id")
node_ids = list(map(lambda item: item['str_id'], tmp_node_ids))
unique_node_ids = set(node_ids)

print(len(node_ids), len(unique_node_ids))

tmp_relation_ids = base_kg_model.graph_struct.db_conn.execute_query("MATCH (a)-[rel]->(b) RETURN rel.str_id as str_id")
relation_ids = list(map(lambda item: item['str_id'], tmp_relation_ids))
unique_relation_ids = set(relation_ids)

print(len(relation_ids), len(unique_relation_ids))

49597 49597
182838 44328


In [24]:
ne_node_ids = []
ind_node_ids = []
for node_id in tqdm(unique_node_ids):
    new_n_exist = embd_model.vectordbs['nodes'].item_exist(node_id)
    old_n_exist = base_kg_model.embeddings_struct.vectordbs['nodes'].item_exist(node_id)
    if not new_n_exist or not old_n_exist:
        print("not_exist: ", node_id)
        ne_node_ids.append(node_id)

    new_d = embd_model.vectordbs['nodes'].read([node_id])
    old_d = base_kg_model.embeddings_struct.vectordbs['nodes'].read([node_id])
    if new_d[0].document != old_d[0].document:
        print("inv doc: ", node_id)
        ind_node_ids.append(node_id)

print(len(ne_node_ids), len(ind_node_ids))

100%|██████████| 49597/49597 [07:58<00:00, 103.60it/s]

0 0


In [ ]:
ne_t_ids = []
ind_t_ids = []
inid_t_ids = []
for relation_id in tqdm(unique_relation_ids):
    new_t_exist = embd_model.vectordbs['triplets'].item_exist(relation_id)
    old_t_exist = base_kg_model.embeddings_struct.vectordbs['triplets'].item_exist(relation_id)
    if not new_t_exist or not old_t_exist:
        print("not_exist: ", relation_id)
        ne_t_ids.append(relation_id)

    new_d = embd_model.vectordbs['triplets'].read([relation_id], includes=['documents', 'metadatas'])
    old_d = base_kg_model.embeddings_struct.vectordbs['triplets'].read([relation_id])
    if new_d[0].document != old_d[0].document:
        print("inv doc: ", relation_id)
        ind_t_ids.append(relation_id)

    t_id = new_d[0].metadata['t_id']
    t_exist = base_kg_model.graph_struct.db_conn.item_exist(t_id)
    if not t_exist:
        print("t_id not exist: ", relation_id)
        inid_t_ids.append(relation_id)

    base_triplet = base_kg_model.graph_struct.db_conn.read([t_id])[0]
    if base_triplet.relation.id != relation_id:
        print("inv str_id: ", relation_id)
        inid_t_ids.append(relation_id)

print(len(ne_t_ids), len(ind_t_ids), len(inid_t_ids))